# Run PESTPP-IES in an automated bias minimization workflow

Here we will combine prior-data conflict resolution, total error covariance and automatic adaptive localization.  The trifecta!

In [ ]:
%matplotlib inline
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams['font.size']=12
import flopy
import pyemu
%matplotlib inline

## SUPER IMPORTANT: SET HOW MANY PARALLEL WORKERS TO USE

In [ ]:
num_workers = 5

In [ ]:
t_d = "template_history"
m_d = "master_ies_bmw"

In [ ]:
m = flopy.modflow.Modflow.load("freyberg.nam",model_ws=t_d,check=False,forgive=False)

In [ ]:
# plot some model attributes
fig = plt.figure(figsize=(12,7))
ax = plt.subplot(111,aspect="equal")
mm = flopy.plot.PlotMapView(model=m)
mm.plot_grid()
mm.plot_ibound()
mm.plot_bc('SFR')
mm.plot_bc("GHB")
ax = mm.ax
#m.wel.stress_period_data.plot(ax=ax,mflay=2)

# plot obs locations
obs = pd.read_csv(os.path.join("..","base_model_files","obs_loc.csv"))
                  
obs_x = [m.sr.xcentergrid[r-1,c-1] for r,c in obs.loc[:,["row","col"]].values]
obs_y = [m.sr.ycentergrid[r-1,c-1] for r,c in obs.loc[:,["row","col"]].values]
ax.scatter(obs_x,obs_y,marker='.',label="water-level obs",s=80)

#plot names on the pumping well locations
wel_data = m.wel.stress_period_data[0]
wel_x = m.sr.xcentergrid[wel_data["i"],wel_data["j"]]
wel_y = m.sr.ycentergrid[wel_data["i"],wel_data["j"]]
for i,(x,y) in enumerate(zip(wel_x,wel_y)):
    ax.scatter([x],[y],color="red",marker="s",s=50)
    #ax.text(x,y,"{0}".format(i+1),ha="center",va="center")

ax.set_ylabel("y(m)")
ax.set_xlabel("x(m)")
plt.show()

In [ ]:
pst = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
#pst.write_par_summary_table(filename="none")

So we run PESTPP-IES once with `autoadaloc`, PDC resolution and save the resulting posterior residual covariance matrix

In [ ]:
#pst.pestpp_options = {}
pst.pestpp_options["ies_num_reals"] = 50  # enough?
pst.pestpp_options["ies_par_en"] = "prior.jcb"
pst.pestpp_options["ies_bad_phi_sigma"] = 1.75
pst.pestpp_options["overdue_giveup_fac"] = 1.5
# the time to calc each lambda upgrade is substantial, so just use one...
pst.pestpp_options["ies_lambda_mults"] = 1.0
pst.pestpp_options["ies_save_rescov"] = True
pst.control_data.noptmax = 1
pst.pestpp_options["ies_drop_conflicts"] = True
pst.pestpp_options["ies_pdc_sigma_distance"] = 2.0
pst.pestpp_options["ies_no_noise"] = True
pst.pestpp_options['ies_num_threads'] = 3
pst.pestpp_options["ies_autoadaloc"] = True
pst.write(os.path.join(t_d,"freyberg_ies.pst"))


In [ ]:
pst.write(os.path.join(t_d,"freyberg_ies.pst"))
pyemu.os_utils.start_workers(t_d,"pestpp-ies","freyberg_ies.pst",num_workers=num_workers,master_dir=m_d)

In [ ]:
oe_pr = pd.read_csv(os.path.join(m_d,"freyberg_ies.0.obs.csv"),index_col=0)
oe_pt = pd.read_csv(os.path.join(m_d,"freyberg_ies.{0}.obs.csv".format(pst.control_data.noptmax)),index_col=0)
obs = pst.observation_data
fnames = pst.pestpp_options["forecasts"].split(",")
for forecast in fnames:
    fig,ax = plt.subplots(1,1,figsize=(10,5))
    oe_pr.loc[:,forecast].hist(ax=ax,color="0.5",alpha=0.5, label='prior')
    oe_pt.loc[:,forecast].hist(ax=ax,color="b",alpha=0.5, label='posterior')
    ax.plot([obs.loc[forecast,"obsval"],obs.loc[forecast,"obsval"]],ax.get_ylim(),"r", label='truth')
    ax.set_title(forecast)
    ax.legend(loc='upper right')
    
    plt.show()

In [ ]:
rw_obs = pd.read_csv(os.path.join(m_d,"freyberg_ies.adjusted.obs_data.csv"),index_col=0)
rw_obs = rw_obs.weight.to_dict()
nz_obs = pst.observation_data.loc[pst.nnz_obs_names,:].copy()
nz_obs.loc[:,"datetime"] = pd.to_datetime(nz_obs.obsnme.apply(lambda x: x.split("_")[-1]))
pst_base = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
oe_base = pd.read_csv(os.path.join(m_d,"freyberg_ies.obs+noise.csv"),index_col=0)
for nz_group in pst.nnz_obs_groups:
    nz_obs_group = nz_obs.loc[nz_obs.obgnme==nz_group,:]
    fig,ax = plt.subplots(1,1,figsize=(10,2))
    ax.plot(nz_obs_group.datetime,nz_obs_group.obsval,"r-")
    pdc_obs = nz_obs_group.loc[nz_obs_group.obsnme.apply(lambda x: rw_obs[x]==0),:]
   
    ax.scatter(pdc_obs.datetime,pdc_obs.obsval,marker='.',s=50,zorder=20,color='k',ls='-',lw=2,fc="k")
    [ax.plot(nz_obs_group.datetime,oe_pr.loc[r,nz_obs_group.obsnme],color="0.5",lw=0.1) for r in oe_pr.index]
    [ax.plot(nz_obs_group.datetime,oe_pt.loc[r,nz_obs_group.obsnme],color="b",lw=0.1,alpha=0.5) for r in oe_pt.index]
    #[ax.plot(nz_obs_group.datetime,oe_base.loc[r,nz_obs_group.obsnme],color="r",lw=0.1,alpha=0.5) for r in oe_base.index]
    mn = oe_base.loc[:,nz_obs_group.obsnme].min()
    mx = oe_base.loc[:,nz_obs_group.obsnme].max()
    
    #ax.fill_between(nz_obs_group.datetime,mn,mx,fc="r",alpha=0.15)
    ax.set_title(nz_group)
    #vmin = min(nz_obs_group.obsval.min(),oe_pt.loc[:,nz_obs_group.obsnme].min().min())
    #vmax = max(nz_obs_group.obsval.max(),oe_pt.loc[:,nz_obs_group.obsnme].max().max())
    vmin = nz_obs_group.obsval.min() * 0.9
    vmax = nz_obs_group.obsval.max() * 1.1
    ax.set_ylim(vmin,vmax)
plt.show()

In [ ]:
# rw_obs = pd.read_csv(os.path.join(m_d,"freyberg_ies.adjusted.obs_data.csv"),index_col=0)
# rw_obs = rw_obs.weight.to_dict()
# pst.observation_data.loc[:,"weight"] = [rw_obs[oname] for oname in pst.obs_names]
res_cov_file = os.path.join(m_d,"freyberg_ies.{0}.shrunk_res.cov".format(pst.control_data.noptmax))
assert os.path.exists(res_cov_file)
res_cov = pyemu.Cov.from_ascii(res_cov_file)
minvar = ((1./obs.loc[res_cov.names,"weight"])**2).min()
shrink = np.zeros(res_cov.shape)
np.fill_diagonal(shrink,minvar)
lamb = 2. / (oe_pt.shape[0] + 1)
lamb = 0.1
print(lamb)
shrunk = (lamb * shrink) + ((1.-lamb) * res_cov.x)
shrunk = pyemu.Cov(x=shrunk,names=res_cov.names)
shrunk.to_ascii(os.path.join(t_d,"shrunk_obs.cov"))
x = shrunk.to_pearson().x.copy()
x[x==0.0] = np.NaN
plt.imshow(x,cmap="plasma")


In [ ]:

pst.pestpp_options["ies_no_noise"] = False
pst.pestpp_options["ies_drop_conflicts"] = False
pst.pestpp_options["obscov"] = "shrunk_obs.cov"
pst.pestpp_options["ies_group_draws"] = False
pst.write(os.path.join(t_d,"freyberg_ies.pst"))

In [ ]:
pyemu.os_utils.start_workers(t_d,"pestpp-ies","freyberg_ies.pst",num_workers=num_workers,master_dir=m_d)

In [ ]:
oe_pr_last = oe_pr.copy()
oe_pt_last = oe_pt.copy()
oe_pr = pd.read_csv(os.path.join(m_d,"freyberg_ies.0.obs.csv"),index_col=0)
oe_pt = pd.read_csv(os.path.join(m_d,"freyberg_ies.{0}.obs.csv".format(pst.control_data.noptmax)),index_col=0)
obs = pst.observation_data
fnames = pst.pestpp_options["forecasts"].split(",")
for forecast in fnames:
    fig,[ax,ax_last] = plt.subplots(1,2,figsize=(10,5))
    oe_pr.loc[:,forecast].hist(ax=ax,color="0.5",alpha=0.5, label='prior')
    oe_pt.loc[:,forecast].hist(ax=ax,color="b",alpha=0.5, label='posterior')
    ax.plot([obs.loc[forecast,"obsval"],obs.loc[forecast,"obsval"]],ax.get_ylim(),"r", label='truth')
    ax.set_title(forecast)
    ax.legend(loc='upper right')
    
    oe_pr_last.loc[:,forecast].hist(ax=ax_last,color="0.5",alpha=0.5, label='prior')
    oe_pt_last.loc[:,forecast].hist(ax=ax_last,color="b",alpha=0.5, label='posterior')
    ax_last.plot([obs.loc[forecast,"obsval"],obs.loc[forecast,"obsval"]],ax.get_ylim(),"r", label='truth')
    ax_last.set_title("last " + forecast)
    ax.legend(loc='upper right')
    plt.show()

In [ ]:
nz_obs = pst.observation_data.loc[pst.nnz_obs_names,:].copy()
nz_obs.loc[:,"datetime"] = pd.to_datetime(nz_obs.obsnme.apply(lambda x: x.split("_")[-1]))
pst_base = pyemu.Pst(os.path.join(t_d,"freyberg.pst"))
oe_base = pd.read_csv(os.path.join(m_d,"freyberg_ies.obs+noise.csv"),index_col=0)
for nz_group in pst.nnz_obs_groups:
    nz_obs_group = nz_obs.loc[nz_obs.obgnme==nz_group,:]
    fig,ax = plt.subplots(1,1,figsize=(10,2))
    ax.plot(nz_obs_group.datetime,nz_obs_group.obsval,"r-")
    pdc_obs = nz_obs_group.loc[nz_obs_group.obsnme.apply(lambda x: rw_obs[x]==0),:]
   
    ax.scatter(pdc_obs.datetime,pdc_obs.obsval,marker='.',s=50,zorder=20,color='k',ls='-',lw=2,fc="k")
    [ax.plot(nz_obs_group.datetime,oe_pr.loc[r,nz_obs_group.obsnme],color="0.5",lw=0.1) for r in oe_pr.index]
    [ax.plot(nz_obs_group.datetime,oe_pt.loc[r,nz_obs_group.obsnme],color="b",lw=0.1,alpha=0.5) for r in oe_pt.index]
    #[ax.plot(nz_obs_group.datetime,oe_base.loc[r,nz_obs_group.obsnme],color="r",lw=0.1,alpha=0.5) for r in oe_base.index]
    mn = oe_base.loc[:,nz_obs_group.obsnme].min()
    mx = oe_base.loc[:,nz_obs_group.obsnme].max()
    
    #ax.fill_between(nz_obs_group.datetime,mn,mx,fc="r",alpha=0.15)
    ax.set_title(nz_group)
    #vmin = min(nz_obs_group.obsval.min(),oe_pt.loc[:,nz_obs_group.obsnme].min().min())
    #vmax = max(nz_obs_group.obsval.max(),oe_pt.loc[:,nz_obs_group.obsnme].max().max())
    vmin = nz_obs_group.obsval.min() * 0.9
    vmax = nz_obs_group.obsval.max() * 1.1
    ax.set_ylim(vmin,vmax)
plt.show()